# Demo for working with the pid4cat API v2 from Python

Details for the endpoints provided by the API can be found in the [auto-generated documentation](https://api.nfdi4cat.org/testpid/v2/index.html).

In [1]:
import os
import httpx

The username and password are read from environment variables. The required credentials are supplied by HLRS.

In [2]:
api_user = os.environ["NFDI4CAT_PID_API_USER"]
password = os.environ["NFDI4CAT_PID_API_PASSWORD"]

test_api_base_url = "https://api.nfdi4cat.org/testpid/v2/4cat"

Setup a httpx-client with REST API authentication

In [3]:
pid_client = httpx.Client(auth=(api_user, password), timeout=10)

## GET "/" - List all available schemes

Request "Get all the available options" for this api_user.

The term "scheme" is a bit misleading since the allowed values are given by the enum [ResourceCategory](https://nfdi4cat.github.io/pid4cat-model/latest/elements/ResourceCategory/) in the pid4cat-model.

In [4]:
res = pid_client.get(test_api_base_url)
display(res.url)
print(f"Statuscode: {res.status_code}")
res.json()

URL('https://api.nfdi4cat.org/testpid/v2/4cat')

Statuscode: 200


{'LIKAT': {'scheme': ['device', 'ontology']}}

Each item in the "scheme" list has its own API endpoints. 

FYI: "LIKAT" is not the api_user here. It is the name of the organisation that is responsible for the name-assignment in the sub-namespace.

## GET "/{scheme}" - List handles registered for a scheme

The description for this endpoint in the API docu is misleading.

It requests all handles created under the selected scheme.

In [5]:
scheme = "device"
res = pid_client.get(f"{test_api_base_url}/{scheme}/")
display(res.url)
print(f"Statuscode: {res.status_code}")
res.json()

URL('https://api.nfdi4cat.org/testpid/v2/4cat/device/')

Statuscode: 200


{'LIKAT': {'device': ['https://hdl.handle.net/api/handles/21.T11978/TEST/1966-BEC9',
   'https://hdl.handle.net/api/handles/21.T11978/TEST/C9K6-C6K9',
   'https://hdl.handle.net/api/handles/21.T11978/TEST/E3E6-A9C1',
   'https://hdl.handle.net/api/handles/21.T11978/TEST/EK1B-EME9',
   'https://hdl.handle.net/api/handles/21.T11978/TEST/2E55-C971',
   'https://hdl.handle.net/api/handles/21.T11978/TEST/MADC-8A79',
   'https://hdl.handle.net/api/handles/21.T11978/TEST/98EB-K3D1']}}

## PUT "/{scheme}/" - Create a new handle in a scheme.

Currently there is no control over the ID-suffix. This has to be added. This should also be a POST request since it creates a resource (the PID).

In [6]:
scheme = "device"  # can be sample, device, substance, ...
target_url = "https://example.org/device_ABC1"
params = {"data": target_url}

handle_create_url = f"{test_api_base_url}/{scheme}/"
res = pid_client.put(handle_create_url, params=params)
print(f"Statuscode: {res.status_code}")
res.json()

Statuscode: 200


{'LIKAT': {'handle': 'https://hdl.handle.net/api/handles/21.T11978/test/k922-5cd1'}}

## GET "/{scheme}/{id}" - Get the target URL (landing page) of a handle in a scheme

Here "id" represents not the full handle-suffix but just the id-part.

In [7]:
scheme = "device"
id_ = "c9k6-c6k9"
res = pid_client.get(f"{test_api_base_url}/{scheme}/{id_}")
display(res.url)
print(f"Statuscode: {res.status_code}")
res.json()

URL('https://api.nfdi4cat.org/testpid/v2/4cat/device/c9k6-c6k9')

Statuscode: 200


{'LIKAT': {'value': 'https://example.org/device_1'}}

## PUT "/{scheme}/{id}" - Update the target URL of a handle in a scheme

In [8]:
scheme = "device"  # can be sample, device, substance, ...
target_url = "https://example.org/device_1"
id_ = "c9k6-c6k9"
params = {"data": target_url}

update_url = f"{test_api_base_url}/{scheme}/{id_}/"
res = pid_client.put(update_url, params=params)
print(f"Statuscode: {res.status_code}")
res.json()

Statuscode: 200


{'LIKAT': {'handle': 'https://hdl.handle.net/api/handles/21.T11978/test/c9k6-c6k9?index=1'}}

## GET "/{scheme}/{id}/metadata/{type}" - Get metadata for a handle by type

In [9]:
scheme = "device"
id_ = "c9k6-c6k9"
type_ = "email"  # "status"
res = pid_client.get(f"{test_api_base_url}/{scheme}/{id_}/metadata/{type_}/")
display(res.url)
print(f"Statuscode: {res.status_code}")
res.json()

URL('https://api.nfdi4cat.org/testpid/v2/4cat/device/c9k6-c6k9/metadata/email/')

Statuscode: 200


{'LIKAT': {'value': 'pids@example.org'}}

## PUT "/{scheme}/{id}/metadata/"

The parameter to hide the metadata is not needed in my opinion. I would be fine with removing it. Its documentation is also misleading: while it is pretty clear that you should set a parameter named "hide" to "false" to make metadata visible, the API description says "Metadata visiblitiy status" and based on this you would select "true" to make metadata visible.

In [10]:
scheme = "device"
id_ = "c9k6-c6k9"
type_ = "email"
params = {"type": type_, "value": "pids@example.org", "hide": "false"}

update_url = f"{test_api_base_url}/{scheme}/{id_}/metadata/"
res = pid_client.put(update_url, params=params)
print(f"Statuscode: {res.status_code}")
res.json()

Statuscode: 200


{'LIKAT': {'handle': 'https://hdl.handle.net/api/handles/21.T11978/test/c9k6-c6k9?index=10'}}

## PUT "/{scheme}/{id}/metadata/{type}/" - Update metadata by type (field)

It is difficult to figure out how to use this endpoint from the API documentation because it lacks a working example for the json expected in the response body.

Below, the Python request is build separately before sending it. This allows to inspect the elements of the send request in detail.

In [12]:
scheme = "device"
id_ = "c9k6-c6k9"
type_ = "resource"
params = {"hide": False}
data = {
    "label": "Resource label",
    "description": "Resource description",
    "resource_category": "DEVICE",
    "representation_variants": [
        {
            "variant_url": "https://example.org/resource",
            "media_type": "text/turtle",
            "encoding_format": "UTF-8",
            "size": 12345,
        }
    ],
}

update_url = f"{test_api_base_url}/{scheme}/{id_}/metadata/{type_}/"
request = pid_client.build_request("PUT", update_url, params=params, json=data)
print(f"Request method: {request.method}")
print(f"Request URL: {request.url}")
print(f"Request headers: {request.headers}")
print(f"Request body: {request.content.decode()}")

res = pid_client.send(request)
print(f"Statuscode: {res.status_code}")
res.text

Request method: PUT
Request URL: https://api.nfdi4cat.org/testpid/v2/4cat/device/c9k6-c6k9/metadata/resource/?hide=false
Request headers: Headers({'host': 'api.nfdi4cat.org', 'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'connection': 'keep-alive', 'user-agent': 'python-httpx/0.28.1', 'content-length': '233', 'content-type': 'application/json'})
Request body: {"label":"Resource label","description":"Resource description","resource_category":"DEVICE","representation_variants":[{"variant_url":"https://example.org/resource","media_type":"text/turtle","encoding_format":"UTF-8","size":12345}]}
Statuscode: 200


'{"LIKAT":{"handle":"https://hdl.handle.net/api/handles/21.T11978/test/c9k6-c6k9?index=14"}}'